In [112]:
from scipy.cluster.vq import *
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC
from sklearn.externals import joblib
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
from imutils import paths
import pickle
from sklearn.metrics import accuracy_score


In [72]:
f = open('train.txt',"r", encoding='utf-8')
lines = f.readlines()
image_paths=[]  
image_classes=[]
for line in lines:
    curLine=line.strip().split(" ")
    image_paths.append(curLine[0]) 
    image_classes.append(curLine[-1])
time=1
allt=len(image_paths)
ims = []
print("loading images")
for image_path in image_paths:
    img  = cv2.imread(image_path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img=cv2.resize(img,(32,32),interpolation=cv2.INTER_CUBIC)
    ims.append(img)
    print(str(time)+"/"+str(allt) +"  " + image_path, end="\r")
    time = time +1
print("\nDone!")

loading images
63325/63325  images/n02172182/n02172182_9821.JPEGGG
Done!


In [73]:
print(len(image_paths))
print(len(image_classes))

63325
63325


In [74]:
sift= cv2.xfeatures2d.SIFT_create()
des_list = []
time=1
allt=len(image_paths)
print("Calculating all the descriptors and keypoints...")
for im in ims:
    kp, des = sift.detectAndCompute(im,None)
    if str(des) != 'None':
        des_list.append((image_paths[time-1], des))       
        print(str(time) + "/" + str(allt), end="\r")
        time = time +1
    else:
        print(image_paths[time-1])
        del image_classes[time-1]
        del image_paths[time-1]
print("\nDone!")

Calculating all the descriptors and keypoints...
images/n02111500/n02111500_7841.JPEG
images/n02111500/n02111500_8916.JPEG
images/n02111889/n02111889_33.JPEG
images/n02111889/n02111889_5070.JPEG
images/n02111889/n02111889_5075.JPEG
images/n02111889/n02111889_6268.JPEG
images/n02112706/n02112706_2456.JPEG
images/n02113023/n02113023_14203.JPEG
images/n02113023/n02113023_8734.JPEG
images/n02113023/n02113023_9093.JPEG
images/n02113186/n02113186_10248.JPEG
images/n02113186/n02113186_4390.JPEG
images/n02113186/n02113186_9594.JPEG
images/n02113624/n02113624_8820.JPEG
images/n02113712/n02113712_14837.JPEG
images/n02113712/n02113712_18789.JPEG
images/n02114367/n02114367_4918.JPEG
images/n02114367/n02114367_6232.JPEG
images/n02114367/n02114367_6850.JPEG
images/n02114548/n02114548_14235.JPEG
images/n02114548/n02114548_599.JPEG
images/n02114548/n02114548_6856.JPEG
images/n02114712/n02114712_19056.JPEG
images/n02114855/n02114855_11211.JPEG
images/n02114855/n02114855_11738.JPEG
images/n02114855/n021

images/n02168699/n02168699_6268.JPEG
images/n02168699/n02168699_7267.JPEG
images/n02168699/n02168699_7288.JPEG
images/n02168699/n02168699_755.JPEG
images/n02169497/n02169497_1147.JPEG
images/n02169497/n02169497_2296.JPEG
images/n02169497/n02169497_3266.JPEG
images/n02169497/n02169497_3531.JPEG
images/n02169497/n02169497_4087.JPEG
images/n02169497/n02169497_438.JPEG
images/n02169497/n02169497_6636.JPEG
images/n02169497/n02169497_9535.JPEG
images/n02172182/n02172182_1068.JPEG
images/n02172182/n02172182_1818.JPEG
images/n02172182/n02172182_2313.JPEG
images/n02172182/n02172182_3266.JPEG
images/n02172182/n02172182_4104.JPEG
images/n02172182/n02172182_4172.JPEG
images/n02172182/n02172182_4408.JPEG
images/n02172182/n02172182_5259.JPEG
images/n02172182/n02172182_6597.JPEG
images/n02172182/n02172182_7284.JPEG
images/n02172182/n02172182_803.JPEG
images/n02172182/n02172182_8983.JPEG
63083/63325
Done!


In [75]:
# Stack all the descriptors vertically in a numpy array
print("Stacking all the descriptors in a numpy array...")
time = 1
allt = len(des_list)
descriptors = des_list[0][1]
for image_path,descriptor in des_list[1:]:
    descriptors = np.vstack((descriptors, descriptor))
    print(str(time)+"/"+str(allt), end="\r")
    time+=1
print("\nDone!")

Stacking all the descriptors in a numpy array...
63082/63083
Done!


In [76]:
# Perform k-means clustering
k = 128
voc, variance = kmeans(descriptors, k, 1)
print("Done doing K-means")

Done doing K-means


In [77]:
# Create the histogram itself of the image features
im_features = np.zeros((len(image_paths), k), "float32")
for i in range(len(image_paths)):
    print("Calculating distance for image "+str(i)+"...", end="\r")
    words, distance = vq(des_list[i][1],voc)
    for w in words:
        im_features[i][w] += 1

In [78]:
nbr_occurences = np.sum( (im_features > 0) * 1, axis = 0)
idf = np.array(np.log((1.0*len(image_paths) + 1) / (1.0*nbr_occurences + 1)), 'float32')
scaled_im_features = im_features * idf

In [79]:
stdSlr = StandardScaler().fit(scaled_im_features)
scaled_im_features = stdSlr.transform(scaled_im_features)

In [80]:
siftclf = LinearSVC()
siftclf.fit(scaled_im_features, np.array(image_classes))

C:\Users\rep03\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [81]:
g = open('val.txt',"r", encoding='utf-8')
lines_val = g.readlines()
image_paths_val=[]  
image_classes_val=[]
for line1 in lines_val:
    curLine_val=line1.strip().split(" ")
    image_paths_val.append(curLine_val[0]) 
    image_classes_val.append(curLine_val[-1])
time1=1
allt1=len(image_paths_val)
ims_val = []
print("loading images")
for image_path1 in image_paths_val:
    img1  = cv2.imread(image_path1)
    img1=cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    img1=cv2.resize(img1,(32,32),interpolation=cv2.INTER_CUBIC)
    ims_val.append(img1)
    print(str(time1)+"/"+str(allt1) +"  " + image_path1, end="\r")
    time1 = time1 +1
print("\nDone!")

loading images
450/450  images/n02172182/n02172182_9982.JPEG
Done!


In [82]:
print(len(image_paths_val))
print(len(image_classes_val))

450
450


In [83]:
sift= cv2.xfeatures2d.SIFT_create()
des_list1 = []
time1=1
allt1=len(image_paths_val)
print("Calculating all the descriptors and keypoints...")
for im1 in ims_val:
    kp, des = sift.detectAndCompute(im1,None)
    if str(des) != 'None':
        des_list1.append((image_paths_val[time1-1], des))       
        print(str(time1) + "/" + str(allt1), end="\r")
        time1 = time1 +1
    else:
        print(image_paths_val[time1-1])
        del image_classes_val[time1-1]
        del image_paths_val[time1-1]
print("\nDone!")

Calculating all the descriptors and keypoints...
images/n02115913/n02115913_9926.JPEG
images/n02120079/n02120079_995.JPEG
448/450
Done!


In [84]:
# Stack all the descriptors vertically in a numpy array
print("Stacking all the descriptors in a numpy array...")
time1 = 1
allt1 = len(des_list1)
descriptors_val = des_list1[0][1]
for image_path1,descriptor1 in des_list1[1:]:
    descriptors_val = np.vstack((descriptors_val, descriptor1))
    print(str(time1)+"/"+str(allt1), end="\r")
    time1+=1
print("\nDone!")

Stacking all the descriptors in a numpy array...
447/448
Done!


In [85]:
# Perform k-means clustering
k = 128
voc, variance = kmeans(descriptors_val, k, 1)
print("Done doing K-means")

Done doing K-means


In [86]:
# Create the histogram itself of the image features
im_features_val = np.zeros((len(image_paths_val), k), "float32")
for j in range(len(image_paths_val)):
    print("Calculating distance for image "+str(j)+"...", end="\r")
    words, distance = vq(des_list1[j][1],voc)
    for s in words:
        im_features_val[j][s] += 1

Calculating distance for image 256...

In [87]:
nbr_occurences_val = np.sum( (im_features_val > 0) * 1, axis = 0)
idf = np.array(np.log((1.0*len(image_paths_val) + 1) / (1.0*nbr_occurences_val + 1)), 'float32')
scaled_im_features_val = im_features_val * idf

In [88]:
stdSlr_val = StandardScaler().fit(scaled_im_features_val)
scaled_im_features_val = stdSlr_val.transform(scaled_im_features_val)

In [90]:
h = open('test.txt',"r", encoding='utf-8')
lines_test = h.readlines()
image_paths_test=[]  
image_classes_test=[]
for line2 in lines_test:
    curLine_test=line2.strip().split(" ")
    image_paths_test.append(curLine_test[0]) 
    image_classes_test.append(curLine_test[-1])
time2=1
allt2=len(image_paths_test)
ims_test = []
print("loading images")
for image_path2 in image_paths_test:
    img2  = cv2.imread(image_path2)
    img2=cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    img2=cv2.resize(img2,(32,32),interpolation=cv2.INTER_CUBIC)
    ims_test.append(img2)
    print(str(time2)+"/"+str(allt2) +"  " + image_path2, end="\r")
    time2 = time2 +1
print("\nDone!")

loading images
450/450  images/n02172182/n02172182_981.JPEGG
Done!


In [91]:
print(len(image_paths_test))
print(len(image_classes_test))

450
450


In [92]:
sift= cv2.xfeatures2d.SIFT_create()
des_list2 = []
time2=1
allt2=len(image_paths_test)
print("Calculating all the descriptors and keypoints...")
for im2 in ims_test:
    kp, des = sift.detectAndCompute(im2,None)
    if str(des) != 'None':
        des_list2.append((image_paths_test[time2-1], des))       
        print(str(time2) + "/" + str(allt2), end="\r")
        time2 = time2 +1
    else:
        print(image_paths_test[time2-1])
        del image_classes_test[time2-1]
        del image_paths_test[time2-1]
print("\nDone!")

Calculating all the descriptors and keypoints...
images/n02115641/n02115641_9731.JPEG
images/n02120079/n02120079_9750.JPEG
images/n02172182/n02172182_959.JPEG
447/450
Done!


In [93]:
# Stack all the descriptors vertically in a numpy array
print("Stacking all the descriptors in a numpy array...")
time2 = 1
allt2 = len(des_list2)
descriptors_test = des_list2[0][1]
for image_path2,descriptor2 in des_list2[1:]:
    descriptors_test = np.vstack((descriptors_test, descriptor2))
    print(str(time2)+"/"+str(allt2), end="\r")
    time2+=1
print("\nDone!")

Stacking all the descriptors in a numpy array...
446/447
Done!


In [94]:
# Perform k-means clustering
k = 128
voc, variance = kmeans(descriptors_test, k, 1)
print("Done doing K-means")

Done doing K-means


In [95]:
# Create the histogram itself of the image features
im_features_test = np.zeros((len(image_paths_test), k), "float32")
for l in range(len(image_paths_test)):
    print("Calculating distance for image "+str(l)+"...", end="\r")
    words, distance = vq(des_list2[l][1],voc)
    for t in words:
        im_features_test[l][t] += 1

In [96]:
nbr_occurences_test = np.sum( (im_features_test > 0) * 1, axis = 0)
idf = np.array(np.log((1.0*len(image_paths_test) + 1) / (1.0*nbr_occurences_test + 1)), 'float32')
scaled_im_features_test = im_features_test * idf

In [97]:
stdSlr_test = StandardScaler().fit(scaled_im_features_test)
scaled_im_features_test = stdSlr_test.transform(scaled_im_features_test)

In [99]:
siftclf_test

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [115]:
predict = siftclf.predict(scaled_im_features_test)

In [116]:
print(accuracy_score(image_classes_test,predict))

0.024608501118568233
